# [Django 5.0 Documentation](https://docs.djangoproject.com/en/5.0/intro/overview/), as of April 14, 2024

## [Overview](https://docs.djangoproject.com/en/5.0/intro/overview/)

Meant for full-stack, database-connected web applications such as blogs or news websites, Django provides systems for URL routing, HTML templating, view-generation, and object-relational mapping—translating between table rows in your database and Python class/model objects.

# Installation and file structure

## Creating a project

To create a Django project, you typically create a root folder for your project, cd into the root folder, set up a virtual environment with your preferred version control software, and then [install django](https://docs.djangoproject.com/en/5.0/intro/install/). E.g.,

```bash
# Create and navigate to 'myproject' directory
mkdir myproject && cd myproject
# Create and activate a virtual environment with poetry
poetry init --no-interaction && poetry shell
# Install Django
poetry add django
```

This will install not only the Django Python library, but also the django-admin CLI tool. Now, in the project root directory, we can use it to [set up the file system and development server for our project](https://docs.djangoproject.com/en/5.0/intro/tutorial01/):

```bash
django-admin startproject myproject .
```

This command will create `manage.py` in the project root folder. It will also create a `myproject` subfolder, with `__init.py__`, `asgi.py`, `settings.py`, `urls.py`, and `wsgi.py`.

The structure of a Django project is a bit unusual, because the framework is designed to support multiple applications within a single project. Unlike other frameworks, the `myproject` folder is *not* where you put your application source code! This is instead a dedicated folder for project-level Django admin files that control URL routing (`urls.py`), Django project settings (`settings.py`), and ASGI/WSGI servers.

```
myproject/
├── manage.py
└── myproject/
    ├── __init__.py
    ├── asgi.py
    ├── settings.py
    ├── urls.py
    └── wsgi.py
```

Our source code will instead go in an application folder.

## [Creating an application](https://docs.djangoproject.com/en/5.0/intro/tutorial01/)

Now that we've created our project, we need to set up at least on application folder within the project where our application source code will go. To set up an application folder, we will navigate to the project root folder and use the following terminal command. (Note that `manage.py` is the entry point for project-level Django CLI commands.)

```bash
python manage.py startapp myapp
```

This command creates a `myapp` folder with the files `admin.py`, `apps.py`, `models.py`, `tests.py`, and `views.py`. Tests, templates, and database migrations will mostly be defined at the application level and will go in this `myapp` folder (though any test or template resources that are shared across multiple applications could go in the project root). Thus, the file structure of a single-application Django project might look like this:

```
myproject/
├── manage.py
├── myproject/
│   ├── __init__.py
│   ├── asgi.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
└── myapp/
    ├── __init__.py
    ├── admin.py
    ├── apps.py
    ├── models.py
    ├── tests/
    │   ├── __init__.py
    │   ├── test_models.py
    │   ├── test_views.py
    │   └── test_forms.py
    ├── views.py
    ├── forms.py
    ├── urls.py
    ├── migrations/
    │   └── __init__.py
    └── templates/
        └── index.html
```

While we will discuss URL routing in more detail later, it's important to note here that before you can use your application, you will need to "include" the application-level URL routes in the project-level `url.py` file. To do this, we'd open up `myproject/urls.py` and add a `path` with an `include` function referencing the location of our application's `urls.py` file in the `urlpatterns` list:

In [ ]:
# myproject/urls.py
from django.contrib import admin
from django.urls import include, path # Add `include` import

urlpatterns = [
    path("myapp/", include("myapp.urls")), # Add this path
    path("admin/", admin.site.urls),
]

You will also need to define at least one valid view in `myapp/views.py`. For example,

In [ ]:
# myapp/views.py
from django.http import HttpResponse

def index(request):
    return HttpResponse("Hello, world.")

You should now be able to run your development server with this terminal command:

```bash
python manage.py runserver
```

This will serve a 'Hello, world' page at `http://localhost:8000/myapp/`, which you can navigate to in your web browser.

## [Object-relational mapping in Django](https://docs.djangoproject.com/en/5.0/intro/overview/)

The heart of Django is really its object-relational mapping system.

Much like a Pydantic or sqlalchemy model, you import Django's base `Model` class and any necessary field or constraint classes (e.g., `CharField`, `ForeignKey`, or `CASCADE`) from `django.db.models`. Then then you declare your table row class as a subclass of `Model`, and you define it to match your database schema. Since Django uses the `__str__` method to represent records in logging console outputs, you might also want to overwrite the string method to return some informative field representing the record.

For example,

In [ ]:
from django.db import models

# Object representing a `reporters` table with a single character field
class Reporter(models.Model):
    full_name = models.CharField(max_length=70)

    # Enables Django to print, e.g., `<QuerySet [<Reporter: John Smith>]>` when logging query results
    def __str__(self):
        return self.full_name

# Object representing an `articles` table with 3 fields and cascade-delete fkey constraint
class Article(models.Model):
    pub_date = models.DateField()
    headline = models.CharField(max_length=200)
    content = models.TextField()
    reporter = models.ForeignKey(Reporter, on_delete=models.CASCADE)

The Model class represents the table, whereas individual class instances represent individual rows or records.

Individual rows or records—class instances—may be stored in the `objects` attribute of the class, although you have to explicitly call the `save` method on the instance to do so. For instance, assuming we initiate the record object `r = Reporter(full_name="John Smith")`, we must then save the record to the table with `r.save()`. By default, any new table's primary key is `id`. Given a record `r`, we can get its id with `r.id()`. We can delete it with `r.delete()`.

Once we've saved some records to the table's `objects` attribute, we can use that attribute's powerful lookup API to retrieve records from the table. For instance, `Reporter.objects.all()` returns all records. More commonly, we will use the `objects.get` or `objects.filter` methods, to which we pass arguments of the pattern `field_name__lookup_method=some_value`. The `get` method returns exactly one record (or raises `DoesNotExist` or `MultipleObjectsReturned` exceptions), while `filter` returns a list of records.

In [ ]:
Reporter.objects.get(id=1)
Reporter.objects.get(full_name__startswith="John")
Reporter.objects.filter(full_name__contains="mith")

If we define Django models in `models.py`, we can then generate automatic SQL migrations to run against our database with the `makemigrations` CLI command:

```bash
python manage.py makemigrations
python manage.py migrate
```

We can also generate Django models dynamically from our database or test their consistency with the database with the `inspectdb` extension from `django-extensions`. For instance, to write database models to `models.py`:

```bash
python manage.py inspectdb > models.py
```

## URL routing

In Django, you create a `URLconf` to define your routing system. This maps URL patterns to Python callback functions. For instance, we might have a `news/url.py` file with this `URLconf`:

In [ ]:
from django.urls import path

from . import views

urlpatterns = [
    path("articles/<int:year>/", views.year_archive),
    path("articles/<int:year>/<int:month>/", views.month_archive),
    path("articles/<int:year>/<int:month>/<int:pk>/", views.article_detail),
]

The URL route parameters are retrieved and passed to the callback as function arguments, and the callback function uses these arguments to retrieve and serve a database record. If the user requests `/articles/2005/05/39323/`, Django calls the function `news.views.article_detail(request, year=2005, month=5, pk=39323)`, and the function generates a view for the corresponding article.

## Views

Views in Django are defined by Python callback functions that return either an `HttpResponse` object or an http exception.

Typically, the callback function retrieves information from the database, performs any desired arbitrary transformations using Python code, and then uses that retrieved/transformed data as context to render and return an HTML template.

For instance,

In [ ]:
from django.shortcuts import render

def year_archive(request, year):
    a_list = Article.objects.filter(pub_date__year=year)
    context = {"year": year, "article_list": a_list}
    return render(request, "news/year_archive.html", context)

## HTML Templating

### Templating file system

In your Django settings, you specify a templates directory with `DIRS`. For instance, we might specify a `templates` directory.

In the `return` statement of a view's callback function, you specify the path of the template you want to use to render the view. The path is relative to the `templates` directory specified in `DIRS`.

So if you return a view with `return render(request, "news/year_archive.html", context)`, the view will be rendered with the template located at `news/year_archive.html`.

The templating system is non-mandatory, so you can use a different system if you prefer, although Django templating is well-integrated with the Django object-relational mapping API.

### Template formatting/syntax

- You can load your static assets to a template with `{% load static %}` and access them like `{% static 'images/sitelogo.png' %}`.
- In a template, you surround variables with double braces: `{{ article.headline }}`. 
- You use dots not only to access object attributes, but also to access dictionary values by key and list items by index.
- You use Unix pipes (`|`) to apply an output filter or format tag to a variable (e.g., `{{ article.pub_date|date:"F j, Y" }}` outputs "Text Month, int day, four-digit year" per [PHP date format syntax](https://www.php.net/manual/en/datetime.format.php)). 
- You can chain pipes together, and you can define custom filters and tags that run arbitrary Python code.
- You can do loops like `{% for article in article_list %}<p>{{ article.headline }}</p>{% endfor %}`.

### Template inheritance

- Django uses "template inheritance". At the top of a template, you specify the `DIRS` template to inherit from, like `{% extends "base.html" %}`.
- In the parent template, you define an empty named block like `{% block title %}{% endblock %}`, and in the child template you define a block of the same name and populate it with content to be passed to the parent, like `{% block title %}Articles for {{ year }}{% endblock %}`. For example,

In [ ]:
<!-- templates/base.html -->
{% load static %}
<html>
<head>
    {% block title %}{% endblock %}
</head>
<body></body>
</html>

<!-- templates/child.html -->
{% extends "base.html" %}

{% block title %}
<title>Articles for {{ year }}</title>
{% endblock %}